Creating STARS_AR Data

In [4]:
import pandas as pd
from random import sample
import warnings

from afl_analytics.stars_ar.phase import create_phases, create_match_id_phase
from afl_analytics.stars_ar.ratings import create_action_rating, create_phase_ratings, create_phase_score, get_phases, do_phase_clustering, create_exponential_decay_weights

warnings.filterwarnings('ignore')

In [2]:
actions = pd.read_csv('/arpadl_actions.csv')

Create Phases

Define new phases after:
- Ball goes out of play
- Possession changes over
- 10 seconds or more+

In [3]:
actions = create_phases(actions)

Value Phases

1. Find similar phases to each other using location and Dynamic Time Warping.
2. Phase Rating = proportion of similar phases that end in a goal/score.

In [5]:
actions['match_id_phase'] = create_match_id_phase(actions)

In [ ]:
num_sample = 5000

In [ ]:
sample_match_id_phase = sample(list(actions['match_id_phase'].unique()), num_sample)
sample_match_id_phase.sort()

In [ ]:
sample_actions = actions[actions['match_id_phase'].isin(sample_match_id_phase)]
sample_phases = get_phases(sample_actions)

In [ ]:
labels = do_phase_clustering(sample_phases, n_clusters=20)

In [ ]:
match_id_phase_label_map = dict(zip(sample_match_id_phase, labels))
sample_actions['label'] = sample_actions['match_id_phase'].map(match_id_phase_label_map)

In [ ]:
sample_actions['phase_score'] = create_phase_score(sample_actions)

In [ ]:
sample_actions['phase_rating'] = create_phase_ratings(sample_actions)

Distribute Value over Actions

1. Exponentially weight the actions in a phase. (Latter actions in phase more important)
2. Multiply weight by phase ratign

In [ ]:
sample_actions['weights'] = create_exponential_decay_weights(sample_actions)

In [ ]:
sample_actions['action_rating'] = create_action_rating(sample_actions)

Player Ratings

In [ ]:
player_total_ratings = sample_actions.groupby('player').agg(
    Action_Rating_sum = ('action_rating', 'sum'),
    Action_Rating_count = ('action_rating', 'count')
)
player_total_ratings['Action_Rating_per_Action'] = player_total_ratings['Action_Rating_sum'] / player_total_ratings['Action_Rating_count']

In [ ]:
player_total_ratings.sort_values('Action_Rating_sum', ascending = False).head(10)